# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 65 new papers today
          27 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/26 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2209.08102


extracting tarball to tmp_2209.08102...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.08105


extracting tarball to tmp_2209.08105...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.09088


extracting tarball to tmp_2209.09088...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.09112


extracting tarball to tmp_2209.09112...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.08106-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.08106) | **Limits on the Diffuse Gamma-Ray Background above 10 TeV with HAWC**  |
|| HAWC Collaboration, et al. -- incl., <mark>R. Alfaro</mark>, <mark>R. Diaz Hernandez</mark>, <mark>S. Hernandez</mark>, <mark>A. Lara</mark>, <mark>W.H. Lee</mark>, <mark>J.T. Linnemann</mark> |
|*Appeared on*| *2022-09-20*|
|*Comments*| *8 pages, 3 figures*|
|**Abstract**| The high-energy Diffuse Gamma-Ray Background (DGRB) is expected to be produced by unresolved isotropically distributed astrophysical objects, potentially including dark matter annihilation or decay emissions in galactic or extragalactic structures. The DGRB has only been observed below 1 TeV; above this energy, upper limits have been reported. Observations or stringent limits on the DGRB above this energy could have significant multi-messenger implications, such as constraining the origin of TeV-PeV astrophysical neutrinos detected by IceCube. The High Altitude Water Cherenkov (HAWC) Observatory, located in central Mexico at 4100 m above sea level, is sensitive to gamma rays from a few hundred GeV to several hundred TeV and continuously observes a wide field-of-view (2 sr). With its high-energy reach and large area coverage, HAWC is well-suited to notably improve searches for the DGRB at TeV energies. In this work, strict cuts have been applied to the HAWC dataset to better isolate gamma-ray air showers from background hadronic showers. The sensitivity to the DGRB was then verified using 535 days of Crab data and Monte Carlo simulations, leading to new limits above 10 TeV on the DGRB as well as prospective implications for multi-messenger studies. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.08120-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.08120) | **Exoplanet Imaging Data Challenge, phase II: Characterization of  exoplanet signals in high-contrast images**  |
|| F. Cantalloube, et al. -- incl., <mark>E. Nasedkin</mark>, <mark>M. Samland Ruffio J.-B.i</mark> |
|*Appeared on*| *2022-09-20*|
|*Comments*| *Submitted to SPIE Astronomical Telescopes + Instrumentation 2022, Adaptive Optics Systems VIII, Paper 12185-4*|
|**Abstract**| Today, there exists a wide variety of algorithms dedicated to high-contrast imaging, especially for the detection and characterisation of exoplanet signals. These algorithms are tailored to address the very high contrast between the exoplanet signal(s), which can be more than two orders of magnitude fainter than the bright starlight residuals in coronagraphic images. The starlight residuals are inhomogeneously distributed and follow various timescales that depend on the observing conditions and on the target star brightness. Disentangling the exoplanet signals within the starlight residuals is therefore challenging, and new post-processing algorithms are striving to achieve more accurate astrophysical results. The Exoplanet Imaging Data Challenge is a community-wide effort to develop, compare and evaluate algorithms using a set of benchmark high-contrast imaging datasets. After a first phase ran in 2020 and focused on the detection capabilities of existing algorithms, the focus of this ongoing second phase is to compare the characterisation capabilities of state-of-the-art techniques. The characterisation of planetary companions is two-fold: the astrometry (estimated position with respect to the host star) and spectrophotometry (estimated contrast with respect to the host star, as a function of wavelength). The goal of this second phase is to offer a platform for the community to benchmark techniques in a fair, homogeneous and robust way, and to foster collaborations. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.08242-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.08242) | **Discovery of a new Local Group Dwarf Galaxy Candidate in UNIONS:  Boötes V**  |
|| <mark>Simon E.T. Smith</mark>, et al. |
|*Appeared on*| *2022-09-20*|
|*Comments*| *13 pages, 7 figures, 3 tables. Submitted to the AAS Journals. Please note that this paper is being submitted in coordination with the work of William Cerny et al. These authors independently discovered this same satellite so our two research groups have coordinated the submission of these discovery papers*|
|**Abstract**| We present the discovery of Bo\"otes V, a new ultra-faint dwarf galaxy candidate. This satellite is detected as a resolved overdensity of stars during an ongoing search for new Local Group dwarf galaxy candidates in the UNIONS photometric dataset. It has a physical half-light radius of 26.9$^{+7.5}_{-5.4}$ pc, a $V$-band magnitude of $-$4.5 $\pm$ 0.4 mag, and resides at a heliocentric distance of approximately 100 kpc. We use Gaia DR3 astrometry to identify member stars, characterize the systemic proper motion, and confirm the reality of this faint stellar system. The brightest star in this system was followed up using Gemini GMOS-N long-slit spectroscopy and is measured to have a metallicity of [Fe/H] $=$ -2.85 $\pm$ 0.10 dex and a heliocentric radial velocity of $v_r$ = 5.1 $\pm$ 13.4 km s$^{-1}$. Bo\"otes V is larger (in terms of scale radius), more distant, and more metal-poor than the vast majority of globular clusters. It is likely that Bo\"otes V is an ultra-faint dwarf galaxy, though future spectroscopic studies will be necessary to definitively classify this object. |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.09037-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.09037) | **Modeling the flare in NGC 1097 from 1991 to 2004 as a tidal disruption  event**  |
|| <mark>Zhang XueGuang</mark> |
|*Appeared on*| *2022-09-20*|
|*Comments*| *5 pages, 3 figures, 1 table, Accepted to be published in MNRAS Letter*|
|**Abstract**| In the Letter, interesting evidence is reported to support a central tidal disruption event (TDE) in the known AGN NGC 1097. Considering the motivations of TDE as one probable origination of emission materials of double-peaked broad emission lines and also as one probable explanation to changing-look AGN, it is interesting to check whether are there clues to support a TDE in NGC 1097, not only a changing-look AGN but also an AGN with double-peaked broad emission lines. Under the assumption that the onset of broad H$\alpha$ emission was due to a TDE, the 13years-long (1991-2004) variability of double-peaked broad H$\alpha$ line flux in NGC 1097 can be well predicted by theoretical TDE model, with a $(1-1.5){\rm M_\odot}$ main-sequence star tidally disrupted by the central BH with TDE model determined mass about $(5-8)\times10^7{\rm M_\odot}$. The results provide interesting evidence to not only support TDE-related origin of double-peaked broad line emission materials but also support TDE as an accepted physical explanation to physical properties of changing-look AGN. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.09041-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.09041) | **A Photometric and Spectroscopic Study of the Short-Period Algol EW  Boötis with a $δ$ Sct Pulsator**  |
|| Hye-Young Kim, et al. -- incl., <mark>Jae Woo Lee</mark> |
|*Appeared on*| *2022-09-20*|
|*Comments*| *27 pages, 8 figures, Accepted for publication in AJ*|
|**Abstract**| In this paper, we present TESS photometry and high-resolution spectra of the short-period Algol EW Boo. We obtained double-lined radial velocities (RVs) from the time-series spectra and measured the effective temperature of the primary star as $T_{\rm{eff,1}}$ = 8560 $\pm$ 118 K. For the orbital period study, we collected all times of minima available for over the last 30 years. It was found that the eclipse timing variation of the system could be represented by a periodic oscillation of 17.6 $\pm$ 0.3 years with a semi-amplitude of 0.0041 $\pm$ 0.0001 d. The orbital and physical parameters were derived by simultaneously analyzing the TESS light and RV curves using the Wilson-Devinney (WD) binary star modeling code. The component masses and radii were showed over 3% precision: $M_{1}$ = 2.67 $\pm $ 0.08 M$_{\odot}$, $M_{2}$ = 0.43 $\pm $ 0.01 M$_{\odot}$, $R_{1}$ = 2.01 $\pm $ 0.02 R$_{\odot}$, and $R_{2}$ = 1.35 $\pm $ 0.01 R$_{\odot}$. Furthermore, multiple frequency analyses were performed for the light-curve residuals from the WD model. As a result, we detected 17 pressure-mode pulsations in the region of 40.15 - 52.37 d$^{-1}$. The absolute dimensions and pulsation characteristics showed that the $\delta$ Sct pulsator was the more massive and hotter primary star of the EW Boo. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.09052-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.09052) | **Double Shells of the Planetary Nebula NGC 7009 Minor Axis**  |
|| <mark>Seong-Jae Lee</mark>, Siek Hyung, Masaaki Otsuka |
|*Appeared on*| *2022-09-20*|
|*Comments*| *15 pages, 9 figures, accepted for publication in ApJ*|
|**Abstract**| We analyzed the minor axis spectra of the elliptical planetary nebula (PN) NGC 7009 observed with the Keck HIRES with a 0.862$"$ $\times$ 10$"$ slit placed at about $\sim$7.5$"$ and 10$"$ away from the center and a 0.862$"$ $\times$ 14$"$ slit at the center. The mean densities derived from the integrated [SII] 6716/6731\r{A}, fluxes along the Keck HIRES slit length indicate a density range of 10$^{3.7}$ to 10$^{4.1}$ cm$^{-3}$, while the local densities derived from the slit spectral images show a large local density variation of about 10$^{2.8}$ - 10$^{4.6}$ cm$^{-3}$: local densities vary substantially more than values integrated over the line of sight. The expansion rates of the main and outer shells obtained by [SII] are about 21.7 and 30.0 kms$^{-1}$, respectively. The kinematic results of the [SII] spectral lines correspond to the outermost regions of the two shells and are not representative of the whole PN but are closely related to the other emission lines observed in the shell gas. We conclude that the density contrast leads to the formation of the inner shell, while the change in ionization state leads to the formation of the outer shell. We suggest that the inner main and outer shells result from two successive major ejections. The physical conditions of the central star must have been different when these shells first formed. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.09088-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.09088) | **The shape of dark matter haloes: results from weak lensing in the  Ultraviolet Near-Infrared Optical Northern Survey (UNIONS)**  |
|| Bailey Robison, et al. -- incl., <mark>Hendrik Hildebrandt</mark> |
|*Appeared on*| *2022-09-20*|
|*Comments*| **|
|**Abstract**| Cold dark matter haloes are expected to be triaxial, and so appear elliptical in projection. We use weak gravitational lensing from the Canada-France Imaging Survey (CFIS) component of the Ultraviolet-Near Infrared Optical Northern Survey (UNIONS) to measure the ellipticity of the dark matter haloes around Luminous Red Galaxies (LRGs) from the Sloan Digital Sky Survey Data Release 7 (DR7) and from the CMASS and LOWZ samples of the Baryon Oscillation Spectroscopic Survey (BOSS), assuming their major axes are aligned with the stellar light. We find that DR7 LRGs with masses $M \sim 2.5\times10^{13} \textrm{M}_{\odot}/h$ have halo ellipticities $e=0.35\pm0.09$. Expressed as a fraction of the galaxy ellipticity, we find $f_h = 1.4\pm0.4$. For BOSS LRGs, the detection is of marginal significance: $e = 0.17\pm0.10$ and $f_h=0.1\pm0.4$. These results are in agreement with other measurements of halo ellipticity from weak lensing and, taken together with previous results, suggest an increase of halo ellipticity of $0.10\pm0.05$ per decade in halo mass. This trend agrees with the predictions from hydrodynamical simulations, which find that at higher halo masses, not only do dark matter haloes become more elliptical, but that the misalignment between major axis of the stellar light in the central galaxy and that of the dark matter decreases. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.09112-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.09112) | **SPECULOOS Northern Observatory: searching for red worlds in the northern  skies**  |
|| Artem Y. Burdanov, et al. -- incl., <mark>Lionel Garcia</mark>, <mark>James McCormac</mark> |
|*Appeared on*| *2022-09-20*|
|*Comments*| *Accepted for publication in PASP (Publications of the Astronomical Society of the Pacific), 13 pages, 9 figures*|
|**Abstract**| SPECULOOS is a ground-based transit survey consisting of six identical 1-m robotic telescopes. The immediate goal of the project is to detect temperate terrestrial planets transiting nearby ultracool dwarfs (late M-dwarf stars and brown dwarfs), which could be amenable for atmospheric research with the next generation of telescopes. Here, we report the developments of the northern counterpart of the project - SPECULOOS Northern Observatory, and present its performance during the first three years of operations from mid-2019 to mid-2022. Currently, the observatory consists of one telescope, which is named Artemis. The Artemis telescope demonstrates remarkable photometric precision, allowing it to be ready to detect new transiting terrestrial exoplanets around ultracool dwarfs. Over the period of the first three years after the installation, we observed 96 objects from the SPECULOOS target list for 6000 hours with a typical photometric precision of $0.5\%$, and reaching a precision of $0.2\%$ for relatively bright non-variable targets with a typical exposure time of 25 sec. Our weather downtime (clouds, high wind speed, high humidity, precipitation and/or high concentration of dust particles in the air) over the period of three years was 30% of overall night time. Our actual downtime is 40% because of additional time loss associated with technical problems. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2209.08106.md
    + _build/html/tmp_2209.08106/./figures/pinc_diff_rd41_dp_gp_fhbin7_gpbin7.png
    + _build/html/tmp_2209.08106/./figures/flux_rd41_gp_pinc_fhbin_gpbin_NB_E25.png
    + _build/html/tmp_2209.08106/./figures/radec_crab3_geminga5_b03_dp_gp_qly_fhbin9_precis_zoom_kInvertedDarkBodyRadiator.png
exported in  _build/html/2209.08120.md
    + _build/html/tmp_2209.08120/./Images_SPIE_injections_sph0.png
    + _build/html/tmp_2209.08120/./Astrometry_b.png
    + _build/html/tmp_2209.08120/./Astrometry_c.png
    + _build/html/tmp_2209.08120/./Photometry_b.png
    + _build/html/tmp_2209.08120/./Photometry_c.png


exported in  _build/html/2209.08242.md
    + _build/html/tmp_2209.08242/./tile+density-stack.png
    + _build/html/tmp_2209.08242/./proposal-cmdselect.png
    + _build/html/tmp_2209.08242/./rhMv.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$</div>



<div id="title">

# Limits on the Diffuse Gamma-Ray Background above 10 TeV with HAWC

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.08106-b31b1b.svg)](https://arxiv.org/abs/2209.08106)<mark>Appeared on: 2022-09-20</mark> - _8 pages, 3 figures_

</div>
<div id="authors">

HAWC Collaboration, et al. -- incl., <mark><mark>R. Alfaro</mark></mark>, <mark><mark>R. Diaz Hernandez</mark></mark>, <mark><mark>S. Hernandez</mark></mark>, <mark><mark>A. Lara</mark></mark>, <mark><mark>W.H. Lee</mark></mark>, <mark><mark>J.T. Linnemann</mark></mark>

</div>
<div id="abstract">

**Abstract:** The high-energy Diffuse Gamma-Ray Background (DGRB) is expected to be produced by unresolved isotropically distributed astrophysical objects, potentially including dark matter annihilation or decay emissions in galactic or extragalactic structures. The DGRB has only been observed below 1 TeV; above this energy, upper limits have been reported. Observations or stringent limits on the DGRB above this energy could have significant multi-messenger implications, such as constraining the origin of TeV-PeV astrophysical neutrinos detected by IceCube. The High Altitude Water Cherenkov (HAWC) Observatory, located in central Mexico at 4100 m above sea level, is sensitive to gamma rays from a few hundred GeV to several hundred TeV and continuously observes a wide field-of-view (2 sr). With its high-energy reach and large area coverage, HAWC is well-suited to notably improve searches for the DGRB at TeV energies. In this work, strict cuts have been applied to the HAWC dataset to better isolate gamma-ray air showers from background hadronic showers. The sensitivity to the DGRB was then verified using 535 days of Crab data and Monte Carlo simulations, leading to new limits above 10 TeV on the DGRB as well as prospective implications for multi-messenger studies.

</div>

<div id="div_fig1">

<img src="tmp_2209.08106/./figures/pinc_diff_rd41_dp_gp_fhbin7_gpbin7.png" alt="Fig1" width="100%"/>

**Figure 1. -** Number of observed and expected events in the Crab RoI with respect to PINCness for the $\hat{E}/\mathcal{B}$ bin $g$/7. The expected number of events is the sum of the simulated gamma events from the Crab plus the $\text{DGRB}_\text{scaled}$, which is the average number of events from the DGRB strip found in  $2.22\times10^{-4}$ sr (the size of the Crab RoI). (*fig:pinc_diff*)

</div>
<div id="div_fig2">

<img src="tmp_2209.08106/./figures/flux_rd41_gp_pinc_fhbin_gpbin_NB_E25.png" alt="Fig2" width="100%"/>

**Figure 2. -** Limits on the DGRB using 535 days of HAWC data compared to the diffuse electron/positron flux observed by HESS \citep{hess1_graph,hess2_graph}. Also shown is the observed DGRB by the Fermi-LAT \citep{fermi_graph}, as well as previous high-energy limits by GRAPES \citep{grapes_graph} and CASA-MIA \citep{casa-mia_graph}. The lines represent several astrophysical models which will be discussed in \emph{Constraining Astrophysical Models}. The gamma-ray flux corresponding to the IceCube $\nu_\mu+\overline{\nu}_\mu$ astrophysical flux \citep{icecube} is shown (see \emph{Constraining Astrophysical Models - IceCube Neutrino Spectrum}) for both $p\gamma$ and $pp$ interactions. (*fig:ul_mc*)

</div>
<div id="div_fig3">

<img src="tmp_2209.08106/./figures/radec_crab3_geminga5_b03_dp_gp_qly_fhbin9_precis_zoom_kInvertedDarkBodyRadiator.png" alt="Fig3" width="100%"/>

**Figure 3. -** Map of the Diffuse Gamma-Ray Background strip, centered on the Crab Nebula's declination. The Crab Nebula, Geminga and the Galactic Plane have been removed, resulting in a 0.57 sr area. (*fig:DGRB_strip*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\tbd}[1]{\textcolor{red}{#1}}$
$\newcommand{\baselinestretch}{1.0}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\tbd$}[1]{\textcolor{red}{#1}}$
$\newcommand{$\baselinestretch$}{1.0}$</div>



<div id="title">

# Exoplanet Imaging Data Challenge, phase II: Characterization of exoplanet signals in high-contrast images

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.08120-b31b1b.svg)](https://arxiv.org/abs/2209.08120)<mark>Appeared on: 2022-09-20</mark> - _Submitted to SPIE Astronomical Telescopes + Instrumentation 2022, Adaptive Optics Systems VIII, Paper 12185-4_

</div>
<div id="authors">

F. Cantalloube, et al. -- incl., <mark><mark>E. Nasedkin</mark></mark>, <mark><mark>M. Samland Ruffio J.-B.i</mark></mark>

</div>
<div id="abstract">

**Abstract:** Today, there exists a wide variety of algorithms dedicated to high-contrast imaging, especially for the detection and characterisation of exoplanet signals. These algorithms are tailored to address the very high contrast between the exoplanet signal(s), which can be more than two orders of magnitude fainter than the bright starlight residuals in coronagraphic images. The starlight residuals are inhomogeneously distributed and follow various timescales that depend on the observing conditions and on the target star brightness. Disentangling the exoplanet signals within the starlight residuals is therefore challenging, and new post-processing algorithms are striving to achieve more accurate astrophysical results.The Exoplanet Imaging Data Challenge is a community-wide effort to develop, compare and evaluate algorithms using a set of benchmark high-contrast imaging datasets. After a first phase ran in 2020 and focused on the detection capabilities of existing algorithms, the focus of this ongoing second phase is to compare the characterisation capabilities of state-of-the-art techniques. The characterisation of planetary companions is two-fold: the astrometry (estimated position with respect to the host star) and spectrophotometry (estimated contrast with respect to the host star, as a function of wavelength).The goal of this second phase is to offer a platform for the community to benchmark techniques in a fair, homogeneous and robust way, and to foster collaborations.

</div>

<div id="div_fig1">

<img src="tmp_2209.08120/./Images_SPIE_injections_sph0.png" alt="Fig2" width="100%"/>

**Figure 2. -** Example injection procedure for the training data set `sphere0'. Top panel: Median of the image cube at the shortest wavelength with the position of the two injections `b' and `c' indicated by colored circles (left) and their injected spectra of the two planetary signals (middle), sampled at the SPHERE-IFS spectral resolving power (right).
    Bottom panel: Airmass factor used for the whole observing sequence (left); extracted spectrum of the non-coronagraphic PSF (middle); stellar SED model used for the target star (right), sampled at the SPHERE-IFS spectral resolving power. (*fig:sph0*)

</div>
<div id="div_fig2">

<img src="tmp_2209.08120/./Astrometry_b.png" alt="Fig5.1" width="50%"/><img src="tmp_2209.08120/./Astrometry_c.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** Astrometry estimations of the injected planets using PCA-NEGFC (orange) and ANDROMEDA (blue), relative to the ground truth (black) in Cartesian coordinates. The size of the symbols corresponds to the 1-sigma uncertainty (averaged in x- and y- direction) and the shaded area to the 3-sigma uncertainty. The two centered grey shaded areas represent the size of a half resolution element ($0.5\lambda/D$) for the shortest and largest wavelengths of the spectro-imager.
    Left: Planet 'b' ($155 \mathrm{mas}$). Right: Planet 'c' ($616 \mathrm{mas}$). (*fig:astro*)

</div>
<div id="div_fig3">

<img src="tmp_2209.08120/./Photometry_b.png" alt="Fig6.1" width="50%"/><img src="tmp_2209.08120/./Photometry_c.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Spectro-photometry estimations of the injected planets using PCA-NEGFC (orange line) and ANDROMEDA (blue line), compared to the ground truth (black line), as well as the residuals normalized by the ground truth (bottom panel). The error bars displayed correspond to the 3-sigma uncertainties on the photometry estimation.
    Top: Planet 'b' (Blackbody spectrum).
    Bottom: Planet 'c' (HR8799e-like spectrum). (*fig:phot*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\warn}[1]{\textcolor{red}{\textbf{#1}}}$
$\newcommand{\plot}[1]{\textit{#1}:}$
$\newcommand{\code}[1]{\texttt{#1}}$
$\newcommand{\rh}{r_{\text{h}}}$
$\newcommand{\ebv}{E(B-V)}$
$\newcommand{\Rv}{R_{V}}$
$\newcommand{\Msun}{\textup{M}_\odot}$
$\newcommand{\Lsun}{\textup{L}_\odot}$
$\newcommand{\sqdeg}{deg^2}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\boov}{Bo\"otes V}$
$\newcommand{\tabspace}{\vspace{2pt}}$
$\newcommand{\ubar}[1]{\underaccent{\bar}{#1}}$
$\newcommand{\ii}{{~\sc ii}}$
$\newcommand{\ie}{{i.e., }}$
$\newcommand{\eg}{{e.g., }}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\warn$}[1]{\textcolor{red}{\textbf{#1}}}$
$\newcommand{$\plot$}[1]{\textit{#1}:}$
$\newcommand{$\code$}[1]{\texttt{#1}}$
$\newcommand{$\rh$}{r_{\text{h}}}$
$\newcommand{$\ebv$}{E(B-V)}$
$\newcommand{$\Rv$}{R_{V}}$
$\newcommand{$\Msun$}{\textup{M}_\odot}$
$\newcommand{$\Lsun$}{\textup{L}_\odot}$
$\newcommand{$\sqdeg$}{deg^2}$
$\newcommand{$\kms$}{km s^{-1}}$
$\newcommand{$\boov$}{Bo\"otes V}$
$\newcommand{$\tabspace$}{\vspace{2pt}}$
$\newcommand{$\ubar$}[1]{\underaccent{\bar}{#1}}$
$\newcommand{$\ii$}{{~\sc ii}}$
$\newcommand{$\ie$}{{i.e., }}$
$\newcommand{$\eg$}{{e.g., }}$</div>



<div id="title">

# tes V

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.08242-b31b1b.svg)](https://arxiv.org/abs/2209.08242)<mark>Appeared on: 2022-09-20</mark> - _13 pages, 7 figures, 3 tables. Submitted to the AAS Journals. Please note that this paper is being submitted in coordination with the work of William Cerny et al. These authors independently discovered this same satellite so our two research groups have coordinated the submission of these discovery papers_

</div>
<div id="authors">

<mark>Simon E.T. Smith</mark>, et al.

</div>
<div id="abstract">

**Abstract:** We present the discovery of$\boov$, a new ultra-faint dwarf galaxy candidate. This satellite is detected as a resolved overdensity of stars during an ongoing search for new Local Group dwarf galaxy candidates in the UNIONS photometric dataset.    It has a physical half-light radius of 26.9$^{+7.5}_{-5.4}$pc, a$V$-band magnitude of$-$4.5$\pm$0.4 mag, and resides at a heliocentric distance of approximately 100 kpc. We use Gaia DR3 astrometry to identify member stars, characterize the systemic proper motion, and confirm the reality of this faint stellar system.    The brightest star in this system was followed up using Gemini GMOS-N long-slit spectroscopy and is measured to have a metallicity of[Fe/H]$=$-2.85$\pm$0.10 dex and a heliocentric radial velocity of$v_r$= 5.1$\pm$13.4$\kms$.$\boov$is larger (in terms of scale radius), more distant, and more metal-poor than the vast majority of globular clusters. It is likely that$\boov$is an ultra-faint dwarf galaxy, though future spectroscopic studies will be necessary to definitively classify this object.

</div>

<div id="div_fig1">

<img src="tmp_2209.08242/./tile+density-stack.png" alt="Fig2" width="100%"/>

**Figure 2. -** $\plot${Left} 8$\arcmin$$\times$ 8$\arcmin$ cutout from CFIS-r imaging tiles. Bright sources with diffraction spikes are likely Milky Way foreground stars. The clustering of stars and light in the centre of this image is $\boov$. $\plot${Middle} Smoothed spatial density plot of all stellar sources on the 8$\arcmin$$\times$ 8$\arcmin$ tile. No stellar population filtering has been applied to the stars in this smoothed distribution. $\plot${Right} Smoothed spatial density plot of all galaxies in the 8$\arcmin$$\times$ 8$\arcmin$ tile, normalized with respect to the stellar density. There are many more galaxies than stars in this field of view, but there is no discernible overdensity. (*fig:tile+den*)

</div>
<div id="div_fig2">

<img src="tmp_2209.08242/./proposal-cmdselect.png" alt="Fig3" width="100%"/>

**Figure 3. -** $\plot${Left} Sky positions of all stars in a 12$\arcmin$$\times$ 12 $\arcmin$ region about $\boov$, projected onto the tangent plane centered at RA, Dec = (14h 15m 38.6s, +32\textdegree 54$\arcmin$ 42$\arcsec$). Dark blue sources are those that meet the isochrone selection criterion (Right). Yellow points are stars identified in Gaia DR3 as sources with complete astrometric information, and are selected as high confidence (probability $>$ 90\%) members by our maximum-likelihood membership selection algorithm (see Section \ref{subsec:gaia}). The concentric black ellipses indicate 1 $\times$, 3 $\times$, and 5 $\times$ the half-light radius ($\rh$) as determined by the MCMC fit (see Section \ref{subsec:structural}).
    $\plot${Right} Color-magnitude diagram of extinction-corrected CFIS-r and Pan-STARRS-i for all stellar sources in a 12$\arcmin$$\times$ 12$\arcmin$ region about $\boov$. We overlay an old (13 Gyr), metal-poor ([Fe/H] = $-$2.3) isochrone, shifted to a distance of 100 kpc. We use a broad color selection criterion, selecting all stars with $(r-i)_0 \leq 0.1$ from the isochrone and we add, in quadrature, the empirical photometric errors from each band. Stars consistent with this selection criterion are highlighted in dark blue.
     (*fig:cmdselect*)

</div>
<div id="div_fig3">

<img src="tmp_2209.08242/./rhMv.png" alt="Fig1" width="100%"/>

**Figure 1. -** Absolute V-band magnitude (M$_V$) vs. half-light radius ($\rh$) plane showing both the dwarf galaxy (black markers) and globular cluster (blue Xs) populations. We show both candidate and spectroscopically confirmed dwarf galaxies within 300 kpc of the Milky Way. $\boov$ is shown as a yellow triangle. (*fig:rhMv*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

38  publications in the last 7 days.
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers